<img align="left" src="https://lever-client-logos.s3.amazonaws.com/864372b1-534c-480e-acd5-9711f850815c-1524247202159.png" width=200>
<br></br>

# Hyperparameter Tuning

## *Data Science Unit 4 Sprint 2 Assignment 4*

## Your Mission, should you choose to accept it...

To hyperparameter tune and extract every ounce of accuracy out of this telecom customer churn dataset: [Available Here](https://lambdaschool-data-science.s3.amazonaws.com/telco-churn/WA_Fn-UseC_-Telco-Customer-Churn+(1).csv)

## Requirements

- Load the data
- Clean the data if necessary (it will be)
- Create and fit a baseline Keras MLP model to the data.
- Hyperparameter tune (at least) the following parameters:
 - batch_size
 - training epochs
 - optimizer
 - learning rate (if applicable to optimizer)
 - momentum (if applicable to optimizer)
 - activation functions
 - network weight initialization
 - dropout regularization
 - number of neurons in the hidden layer
 
 You must use Grid Search and Cross Validation for your initial pass of the above hyperparameters
 
 Try and get the maximum accuracy possible out of this data! You'll save big telecoms millions! Doesn't that sound great?


In [105]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
import category_encoders as ce
from sklearn.preprocessing import StandardScaler

from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.wrappers.scikit_learn import KerasClassifier

from sklearn.model_selection import GridSearchCV
from tensorflow.keras.callbacks import TensorBoard
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.layers import Dropout
from tensorflow.keras import regularizers
from tensorflow.keras.optimizers import Adam, Ftrl, Adamax, SGD
from kerastuner.tuners import RandomSearch

In [4]:
# load data
telecom = pd.read_csv('telecom-customer-churn.csv')

In [5]:
print(telecom.shape)
telecom.head(3)

(7043, 21)


,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,...,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
0,7590-VHVEG,Female,0,Yes,No,1,No,No phone service,DSL,No,...,No,No,No,No,Month-to-month,Yes,Electronic check,29.85,29.85,No
1,5575-GNVDE,Male,0,No,No,34,Yes,No,DSL,Yes,...,Yes,No,No,No,One year,No,Mailed check,56.95,1889.5,No
2,3668-QPYBK,Male,0,No,No,2,Yes,No,DSL,Yes,...,No,No,No,No,Month-to-month,Yes,Mailed check,53.85,108.15,Yes


In [32]:
telecom['Churn'] = [1 if c == 'Yes' else 0 for c in telecom['Churn']]

In [34]:
telecom['Churn']

0       0
1       0
2       1
3       0
4       1
       ..
7038    0
7039    0
7040    0
7041    1
7042    0
Name: Churn, Length: 7043, dtype: int64

In [33]:
telecom.dtypes

gender               object
SeniorCitizen         int64
Partner              object
Dependents           object
tenure                int64
PhoneService         object
MultipleLines        object
InternetService      object
OnlineSecurity       object
OnlineBackup         object
DeviceProtection     object
TechSupport          object
StreamingTV          object
StreamingMovies      object
Contract             object
PaperlessBilling     object
PaymentMethod        object
MonthlyCharges      float64
TotalCharges        float64
Churn                 int64
dtype: object

In [7]:
telecom = telecom.replace('^\s*$', -2, regex=True)
telecom.isnull().sum()

customerID          0
gender              0
SeniorCitizen       0
Partner             0
Dependents          0
tenure              0
PhoneService        0
MultipleLines       0
InternetService     0
OnlineSecurity      0
OnlineBackup        0
DeviceProtection    0
TechSupport         0
StreamingTV         0
StreamingMovies     0
Contract            0
PaperlessBilling    0
PaymentMethod       0
MonthlyCharges      0
TotalCharges        0
Churn               0
dtype: int64

In [8]:
telecom['TotalCharges'] = pd.to_numeric(telecom['TotalCharges'])

In [9]:
telecom = telecom.drop(columns=['customerID'], axis=1)

In [10]:
telecom.isnull().sum()

gender              0
SeniorCitizen       0
Partner             0
Dependents          0
tenure              0
PhoneService        0
MultipleLines       0
InternetService     0
OnlineSecurity      0
OnlineBackup        0
DeviceProtection    0
TechSupport         0
StreamingTV         0
StreamingMovies     0
Contract            0
PaperlessBilling    0
PaymentMethod       0
MonthlyCharges      0
TotalCharges        0
Churn               0
dtype: int64

In [11]:
telecom.columns

Index(['gender', 'SeniorCitizen', 'Partner', 'Dependents', 'tenure',
       'PhoneService', 'MultipleLines', 'InternetService', 'OnlineSecurity',
       'OnlineBackup', 'DeviceProtection', 'TechSupport', 'StreamingTV',
       'StreamingMovies', 'Contract', 'PaperlessBilling', 'PaymentMethod',
       'MonthlyCharges', 'TotalCharges', 'Churn'],
      dtype='object')

In [35]:
features = telecom.columns[:-1]
target = telecom.columns[-1]
features, target

(Index(['gender', 'SeniorCitizen', 'Partner', 'Dependents', 'tenure',
        'PhoneService', 'MultipleLines', 'InternetService', 'OnlineSecurity',
        'OnlineBackup', 'DeviceProtection', 'TechSupport', 'StreamingTV',
        'StreamingMovies', 'Contract', 'PaperlessBilling', 'PaymentMethod',
        'MonthlyCharges', 'TotalCharges'],
       dtype='object'),
 'Churn')

In [36]:
X = telecom[features]
y = telecom[target]
X.shape, y.shape

((7043, 19), (7043,))

In [37]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42, shuffle=True)

In [38]:
(X_train.shape, y_train.shape), (X_test.shape, y_test.shape)

(((5282, 19), (5282,)), ((1761, 19), (1761,)))

In [39]:
categories = telecom[features].select_dtypes(include=['object']).columns
categories

Index(['gender', 'Partner', 'Dependents', 'PhoneService', 'MultipleLines',
       'InternetService', 'OnlineSecurity', 'OnlineBackup', 'DeviceProtection',
       'TechSupport', 'StreamingTV', 'StreamingMovies', 'Contract',
       'PaperlessBilling', 'PaymentMethod'],
      dtype='object')

In [40]:
enc = ce.OrdinalEncoder()


X_train_enc = enc.fit_transform(X_train, y_train)
X_test_enc = enc.transform(X_test)


In [41]:
X_train_enc.shape

(5282, 19)

In [42]:
X_train_enc

,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges
6607,1,0,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,25.30,25.30
2598,2,0,1,2,7,2,2,2,1,1,2,1,1,1,1,1,1,75.15,525.00
2345,2,0,1,1,4,2,2,3,2,2,3,2,2,2,1,1,2,20.05,85.50
4093,2,0,1,2,29,2,3,2,1,1,1,1,1,1,1,1,3,76.00,2215.25
693,2,0,1,2,3,2,3,2,1,1,1,1,1,1,1,1,3,75.10,270.70
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3772,1,0,2,2,1,2,2,2,3,1,1,1,3,3,1,1,1,95.00,95.00
5191,2,0,2,1,23,2,3,1,3,3,2,3,3,3,3,1,3,91.10,2198.30
5226,1,0,2,1,12,2,2,3,2,2,3,2,2,2,1,1,1,21.15,306.05
5390,1,1,1,2,12,2,3,2,1,1,2,1,3,3,1,1,1,99.45,1200.15


In [43]:
scaler = StandardScaler()

X_train_scaled = scaler.fit_transform(X_train_enc)
X_test_scaled = scaler.transform(X_test_enc)
print(X_train_scaled[:10])

[[-0.97309929 -0.43609145 -0.96941854 -1.5265618  -1.28288214 -3.06222308
  -2.06867715 -1.18104467 -0.91809493 -1.03965454 -0.99221753 -0.92219807
  -1.11971672 -1.12523148 -0.83139487 -0.83766059 -1.14934396 -1.31004561
  -0.99706287]
 [ 1.02764436 -0.43609145 -0.96941854  0.65506683 -1.03785653  0.32656014
  -0.51525766  0.17245456 -0.91809493 -1.03965454  0.28886118 -0.92219807
  -1.11971672 -1.12523148 -0.83139487 -0.83766059 -1.14934396  0.34483164
  -0.77679203]
 [ 1.02764436 -0.43609145 -0.96941854 -1.5265618  -1.16036933  0.32656014
  -0.51525766  1.52595379  0.2459419   0.09588212  1.56993989  0.24034374
   0.01070066  0.00513609 -0.83139487 -0.83766059 -0.28945235 -1.48433058
  -0.97052633]
 [ 1.02764436 -0.43609145 -0.96941854  0.65506683 -0.13942926  0.32656014
   1.03816184  0.17245456 -0.91809493 -1.03965454 -0.99221753 -0.92219807
  -1.11971672 -1.12523148 -0.83139487 -0.83766059  0.57043927  0.37304921
  -0.03171942]
 [ 1.02764436 -0.43609145 -0.96941854  0.65506683 -1

In [44]:
X_train_scaled.shape

(5282, 19)

In [55]:
y_test

185     1
2715    0
3825    0
1807    1
132     0
       ..
5845    1
2301    0
5121    0
677     1
6062    0
Name: Churn, Length: 1761, dtype: int64

In [57]:
model = Sequential([
    Dense(6, input_dim=19),
    Dense(2)
])

model.compile(loss='binary_crossentropy', metrics=['accuracy'])

In [58]:
model_object = model.fit(x=X_train_scaled, 
          y=y_train, 
          epochs=10, 
          validation_data=(X_test_scaled, y_test))

Epoch 1/10
166/166 [==============================] - 1s 4ms/step - loss: 4.6622 - accuracy: 0.4250 - val_loss: 4.1593 - val_accuracy: 0.4157
Epoch 2/10
166/166 [==============================] - 0s 3ms/step - loss: 3.7452 - accuracy: 0.4339 - val_loss: 3.2267 - val_accuracy: 0.4287
Epoch 3/10
166/166 [==============================] - 0s 3ms/step - loss: 3.1162 - accuracy: 0.4428 - val_loss: 2.8138 - val_accuracy: 0.4526
Epoch 4/10
166/166 [==============================] - 0s 3ms/step - loss: 2.6643 - accuracy: 0.4587 - val_loss: 2.4912 - val_accuracy: 0.4622
Epoch 5/10
166/166 [==============================] - 0s 3ms/step - loss: 2.4236 - accuracy: 0.4744 - val_loss: 2.3474 - val_accuracy: 0.4747
Epoch 6/10
166/166 [==============================] - 0s 3ms/step - loss: 2.3234 - accuracy: 0.4769 - val_loss: 2.1687 - val_accuracy: 0.4713
Epoch 7/10
166/166 [==============================] - 0s 3ms/step - loss: 2.0939 - accuracy: 0.4786 - val_loss: 1.9158 - val_accuracy: 0.4804
Epoch 

### baseline is 51%

In [125]:
# Function to create model, required for KerasClassifier
def create_model(lr, activation='relu'):
    # create optimizer
    adam = Adam(learning_rate=lr)
    # create model
    act = activation
    model = Sequential()
    model.add(Dense(64, input_dim=19, activation=act))
    model.add(Dense(32, activation=act))
    model.add(Dense(16, activation=act))
    model.add(Dense(8, activation=act))
    model.add(Dense(2, activation=act))
    # Compile model
    model.compile(loss='binary_crossentropy', optimizer=adam, metrics=['accuracy'])
    return model

In [126]:
# create model
model2 = KerasClassifier(build_fn=create_model, verbose=0)

In [92]:
# define the grid search parameters
param_grid = {'lr': [.001, .01, .1, .2, .3, .5],
              'batch_size': [10],
              'epochs': [20]}

# Create Grid Search
grid = GridSearchCV(estimator=model2, param_grid=param_grid, n_jobs=12)
grid_result = grid.fit(X_train_scaled, y_train)

# Report Results
print(f"Best: {grid_result.best_score_} using {grid_result.best_params_}")

Best: 0.7351362347602844 using {'batch_size': 10, 'epochs': 20, 'lr': 0.2}


In [93]:
# define the grid search parameters
param_grid = {'lr': [.2],
              'batch_size': [10, 20, 30, 40, 50],
              'epochs': [20]}

# Create Grid Search
grid = GridSearchCV(estimator=model2, param_grid=param_grid, n_jobs=12)
grid_result = grid.fit(X_train_scaled, y_train)

# Report Results
print(f"Best: {grid_result.best_score_} using {grid_result.best_params_}")


Best: 0.6512347280979156 using {'batch_size': 30, 'epochs': 20, 'lr': 0.2}


In [94]:
param_grid = {'lr': [.2],
              'batch_size': [10, 20, 30, 40, 50],
              'epochs': [20, 50, 100]}

# Create Grid Search
grid = GridSearchCV(estimator=model2, param_grid=param_grid, n_jobs=12)
grid_result = grid.fit(X_train_scaled, y_train)

# Report Results
print(f"Best: {grid_result.best_score_} using {grid_result.best_params_}")

Best: 0.7368407845497131 using {'batch_size': 10, 'epochs': 50, 'lr': 0.2}


In [103]:
# define the grid search parameters
param_grid = {'lr': [.2],
              'batch_size': [10],
              'epochs': [50],
              'activation': ['relu', 'softmax', 'softplus']}

# Create Grid Search
grid = GridSearchCV(estimator=model2, param_grid=param_grid, n_jobs=12)
grid_result = grid.fit(X_train_scaled, y_train)

# Report Results
print(f"Best: {grid_result.best_score_} using {grid_result.best_params_}")

Best: 0.7368407845497131 using {'activation': 'relu', 'batch_size': 10, 'epochs': 50, 'lr': 0.2}


In [95]:
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print(f"Means: {mean}, Stdev: {stdev} with: {param}") 

Means: 0.5504771530628204, Stdev: 0.2317484118798782 with: {'batch_size': 10, 'epochs': 20, 'lr': 0.2}
Means: 0.7368407845497131, Stdev: 0.012716509457793904 with: {'batch_size': 10, 'epochs': 50, 'lr': 0.2}
Means: 0.5016507804393768, Stdev: 0.21311348698165491 with: {'batch_size': 10, 'epochs': 100, 'lr': 0.2}
Means: 0.6390167593955993, Stdev: 0.1921707767743572 with: {'batch_size': 20, 'epochs': 20, 'lr': 0.2}
Means: 0.5504771530628204, Stdev: 0.2317484118798782 with: {'batch_size': 20, 'epochs': 50, 'lr': 0.2}
Means: 0.642422616481781, Stdev: 0.1896603942666927 with: {'batch_size': 20, 'epochs': 100, 'lr': 0.2}
Means: 0.556158971786499, Stdev: 0.23043750153031653 with: {'batch_size': 30, 'epochs': 20, 'lr': 0.2}
Means: 0.5439410030841827, Stdev: 0.23307607753767434 with: {'batch_size': 30, 'epochs': 50, 'lr': 0.2}
Means: 0.5410074532032013, Stdev: 0.23361005640635293 with: {'batch_size': 30, 'epochs': 100, 'lr': 0.2}
Means: 0.7368407845497131, Stdev: 0.012716509457793904 with: {'bat

In [134]:
# best params
param_grid = {'lr': [.2],
              'batch_size': [10],
              'epochs': [50]}

# early stopping
stopping = EarlyStopping(monitor='val_accuracy', min_delta=0.001, patience=10)

model_2 = Sequential([
    Dense(64, activation='relu', input_dim=19),
    Dropout(0.1),
    Dense(32, activation='relu'),
    Dropout(0.1),
    Dense(16, activation='relu'),
    Dropout(0.1),
    Dense(8, activation='relu'),
    Dropout(0.1),
    Dense(2, activation='relu')
])

model_2.compile(loss='binary_crossentropy', optimizer=Adam(learning_rate=0.2), metrics=['accuracy'])

In [138]:
model_object2 = model_2.fit(x=X_train_scaled, 
          y=y_train, 
          epochs=100, 
          batch_size=10,                  
          validation_data=(X_test_scaled, y_test),
          callbacks=[stopping])

Epoch 1/100
529/529 [==============================] - ETA: 0s - loss: 3.0850 - accuracy: 0.80 - ETA: 0s - loss: 4.3615 - accuracy: 0.71 - ETA: 0s - loss: 4.3447 - accuracy: 0.71 - ETA: 0s - loss: 4.2289 - accuracy: 0.72 - ETA: 0s - loss: 4.2162 - accuracy: 0.72 - ETA: 0s - loss: 4.2616 - accuracy: 0.72 - ETA: 0s - loss: 4.2439 - accuracy: 0.72 - ETA: 0s - loss: 4.1917 - accuracy: 0.72 - ETA: 0s - loss: 4.0438 - accuracy: 0.73 - ETA: 0s - loss: 3.9991 - accuracy: 0.74 - ETA: 0s - loss: 4.0133 - accuracy: 0.73 - ETA: 0s - loss: 4.0371 - accuracy: 0.73 - ETA: 0s - loss: 4.0037 - accuracy: 0.74 - ETA: 0s - loss: 4.0056 - accuracy: 0.74 - ETA: 0s - loss: 4.0602 - accuracy: 0.73 - ETA: 0s - loss: 4.0668 - accuracy: 0.73 - 1s 2ms/step - loss: 4.0592 - accuracy: 0.7368 - val_loss: 4.1957 - val_accuracy: 0.7280
Epoch 2/100
529/529 [==============================] - ETA: 0s - loss: 1.5425 - accuracy: 0.90 - ETA: 0s - loss: 4.2308 - accuracy: 0.72 - ETA: 0s - loss: 4.3016 - accuracy: 0.72 - ETA:

In [ ]:
## Random Search

In [141]:
def build_model2(hp):
    model = Sequential()
    model.add(Dense(units=hp.Int('units',
                                        min_value=32,
                                        max_value=512,
                                        step=32),
                           activation='relu'))
    model.add(Dense(64, activation='relu'))
    model.add(Dropout(0.1))
    model.add(Dense(32, activation='relu'))
    model.add(Dropout(0.1))
    model.add(Dense(16, activation='relu'))
    model.add(Dropout(0.1))
    model.add(Dense(8, activation='relu'))
    model.add(Dropout(0.1))
    model.add(Dense(2, activation='relu'))
    model.compile(
        optimizer=Adam(
            hp.Choice('learning_rate',
                      values=[1e-2, 1e-3, 1e-4])),
        loss='binary_crossentropy',
        metrics=['accuracy'])
    
    return model

In [142]:
tuner = RandomSearch(
    build_model2,
    objective='val_accuracy',
    max_trials=5,
    executions_per_trial=3,
    directory='./tuner',
    project_name='hyperpar')

In [143]:
tuner.search_space_summary()

In [146]:
tuner.search(X_train_scaled, y_train,
             epochs=10,
             validation_data=(X_test_scaled, y_test))

Epoch 1/10
166/166 [==============================] - ETA: 0s - loss: 2.1607 - accuracy: 0.34 - ETA: 0s - loss: 0.6536 - accuracy: 0.57 - ETA: 0s - loss: 0.5686 - accuracy: 0.58 - ETA: 0s - loss: 0.5392 - accuracy: 0.57 - ETA: 0s - loss: 0.5304 - accuracy: 0.57 - ETA: 0s - loss: 0.5330 - accuracy: 0.57 - 0s 3ms/step - loss: 0.5209 - accuracy: 0.5613 - val_loss: 0.4459 - val_accuracy: 0.6434
Epoch 2/10
166/166 [==============================] - ETA: 0s - loss: 0.3689 - accuracy: 0.56 - ETA: 0s - loss: 0.4866 - accuracy: 0.62 - ETA: 0s - loss: 0.5059 - accuracy: 0.56 - ETA: 0s - loss: 0.4975 - accuracy: 0.57 - ETA: 0s - loss: 0.4926 - accuracy: 0.57 - ETA: 0s - loss: 0.4845 - accuracy: 0.58 - 0s 2ms/step - loss: 0.4829 - accuracy: 0.5831 - val_loss: 0.4434 - val_accuracy: 0.6099
Epoch 3/10
166/166 [==============================] - ETA: 0s - loss: 0.5270 - accuracy: 0.53 - ETA: 0s - loss: 0.4544 - accuracy: 0.57 - ETA: 0s - loss: 0.4682 - accuracy: 0.60 - ETA: 0s - loss: 0.4672 - accurac

Epoch 1/10
166/166 [==============================] - ETA: 0s - loss: 0.9444 - accuracy: 0.71 - ETA: 0s - loss: 1.6389 - accuracy: 0.70 - ETA: 0s - loss: 1.4746 - accuracy: 0.70 - ETA: 0s - loss: 1.3085 - accuracy: 0.70 - ETA: 0s - loss: 1.1781 - accuracy: 0.70 - ETA: 0s - loss: 1.1073 - accuracy: 0.70 - ETA: 0s - loss: 1.0419 - accuracy: 0.69 - 0s 3ms/step - loss: 1.0121 - accuracy: 0.6982 - val_loss: 0.5883 - val_accuracy: 0.7206
Epoch 2/10
166/166 [==============================] - ETA: 0s - loss: 0.8760 - accuracy: 0.65 - ETA: 0s - loss: 0.7986 - accuracy: 0.68 - ETA: 0s - loss: 0.7682 - accuracy: 0.68 - ETA: 0s - loss: 0.7568 - accuracy: 0.67 - ETA: 0s - loss: 0.7589 - accuracy: 0.66 - ETA: 0s - loss: 0.7305 - accuracy: 0.66 - 0s 2ms/step - loss: 0.7298 - accuracy: 0.6672 - val_loss: 0.5252 - val_accuracy: 0.6604
Epoch 3/10
166/166 [==============================] - ETA: 0s - loss: 0.5565 - accuracy: 0.68 - ETA: 0s - loss: 0.6989 - accuracy: 0.64 - ETA: 0s - loss: 0.6655 - accurac

Epoch 1/10
166/166 [==============================] - ETA: 0s - loss: 3.0540 - accuracy: 0.65 - ETA: 0s - loss: 3.5344 - accuracy: 0.47 - ETA: 0s - loss: 3.2649 - accuracy: 0.52 - ETA: 0s - loss: 3.2134 - accuracy: 0.55 - ETA: 0s - loss: 3.1585 - accuracy: 0.57 - ETA: 0s - loss: 2.9454 - accuracy: 0.57 - 0s 3ms/step - loss: 2.7560 - accuracy: 0.5670 - val_loss: 0.6668 - val_accuracy: 0.5156
Epoch 2/10
166/166 [==============================] - ETA: 0s - loss: 0.5257 - accuracy: 0.46 - ETA: 0s - loss: 0.7760 - accuracy: 0.58 - ETA: 0s - loss: 0.8140 - accuracy: 0.61 - ETA: 0s - loss: 0.7994 - accuracy: 0.59 - ETA: 0s - loss: 0.8216 - accuracy: 0.59 - ETA: 0s - loss: 0.8125 - accuracy: 0.61 - 0s 2ms/step - loss: 0.7968 - accuracy: 0.6189 - val_loss: 0.5754 - val_accuracy: 0.6616
Epoch 3/10
166/166 [==============================] - ETA: 0s - loss: 0.6158 - accuracy: 0.62 - ETA: 0s - loss: 0.5642 - accuracy: 0.61 - ETA: 0s - loss: 0.5913 - accuracy: 0.61 - ETA: 0s - loss: 0.6128 - accurac

Epoch 1/10
166/166 [==============================] - ETA: 0s - loss: 1.3850 - accuracy: 0.37 - ETA: 0s - loss: 0.8405 - accuracy: 0.52 - ETA: 0s - loss: 0.7327 - accuracy: 0.40 - ETA: 0s - loss: 0.6873 - accuracy: 0.36 - ETA: 0s - loss: 0.6704 - accuracy: 0.39 - ETA: 0s - loss: 0.6351 - accuracy: 0.38 - ETA: 0s - loss: 0.6157 - accuracy: 0.38 - 0s 3ms/step - loss: 0.6156 - accuracy: 0.3904 - val_loss: 0.5279 - val_accuracy: 0.7280
Epoch 2/10
166/166 [==============================] - ETA: 0s - loss: 0.5596 - accuracy: 0.65 - ETA: 0s - loss: 0.5328 - accuracy: 0.73 - ETA: 0s - loss: 0.5239 - accuracy: 0.72 - ETA: 0s - loss: 0.5630 - accuracy: 0.65 - ETA: 0s - loss: 0.5676 - accuracy: 0.61 - ETA: 0s - loss: 0.5672 - accuracy: 0.60 - 0s 2ms/step - loss: 0.5685 - accuracy: 0.6079 - val_loss: 0.5537 - val_accuracy: 0.4918
Epoch 3/10
166/166 [==============================] - ETA: 0s - loss: 0.3706 - accuracy: 0.40 - ETA: 0s - loss: 0.7326 - accuracy: 0.47 - ETA: 0s - loss: 0.7195 - accurac

INFO:tensorflow:Oracle triggered exit


In [147]:
tuner.results_summary()

## Stretch Goals:

- Try to implement Random Search Hyperparameter Tuning on this dataset
- Try to implement Bayesian Optimiation tuning on this dataset using hyperas or hyperopt (if you're brave)
- Practice hyperparameter tuning other datasets that we have looked at. How high can you get MNIST? Above 99%?
- Study for the Sprint Challenge
 - Can you implement both perceptron and MLP models from scratch with forward and backpropagation?
 - Can you implement both perceptron and MLP models in keras and tune their hyperparameters with cross validation?